In [1]:
import requests
from bs4 import BeautifulSoup
import random
import pandas as pd
from datetime import datetime


In [2]:
# Parámetros de búsqueda
title = "analista de datos"
location = "UniteStates"

In [3]:
list_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=analista%2Bde%2Bdatos&location=UnitesStates&start=50" #editar link 
response = requests.get(list_url)

#https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search?keywords=analista%2Bde%2Bdatos&location=colombia&geoId=&trk=public_jobs_jobs-search-bar_search-submit&start=25

list_data = response.text 
list_soup = BeautifulSoup(list_data, "html.parser")
page_jobs = list_soup.find_all("li")

In [4]:
# URL base para la paginación
base_url = "https://www.linkedin.com/jobs-guest/jobs/api/seeMoreJobPostings/search"
id_list = []

# Lógica para paginar y obtener más IDs
for start in range(0, 200, 25):  # Cambiar 200 según el número máximo deseado
    list_url = f"{base_url}?keywords={title.replace(' ', '%2B')}&location={location}&start={start}"
    response = requests.get(list_url)
    if response.status_code != 200:
        print(f"Error al obtener datos de la página con inicio {start}")
        break
    list_data = response.text
    list_soup = BeautifulSoup(list_data, "html.parser")
    page_jobs = list_soup.find_all("li")

    # Extraer IDs de los trabajos
    for job in page_jobs:
        try:
            base_card_div = job.find("div", {"class": "base-card"})
            if base_card_div:
                job_id = base_card_div.get("data-entity-urn").split(":")[3]
                id_list.append(job_id)
        except AttributeError:
            continue

# Limitar el número de IDs a procesar si es necesario
id_list = list(set(id_list))  # Eliminar duplicados
print(f"Se encontraron {len(id_list)} IDs únicos.")

# Lista para almacenar información de trabajos
job_list = []
fecha_actual = datetime.now().strftime("%Y-%m-%d")

# Obtener información de cada trabajo
for job_id in id_list:
    # URLs de la vacante
    job_url_api = f"https://www.linkedin.com/jobs-guest/jobs/api/jobPosting/{job_id}"
    job_url_decent = f"https://www.linkedin.com/jobs/view/{job_id}"  # URL limpia

    # Solicitar datos del trabajo
    job_response = requests.get(job_url_api)
    if job_response.status_code != 200:
        continue  # Saltar si hay error en la solicitud

    # Parsear datos del trabajo
    job_soup = BeautifulSoup(job_response.text, "html.parser")
    job_post = {
        "Estado": "Postulado",
        "Posición": None,
        "Fecha de Postulación": fecha_actual,
        "Plataforma": "LinkedIn",
        "Empresa": None,
        "Rubro": None,
        "Nombre del Reclutador": None,
        "Link Posición": job_url_decent,
        "Link Empresa": None,
        "Link Reclutador": None,
    }

    # Extraer detalles del trabajo
    try:
        job_post["Posición"] = job_soup.find("h2", {"class": "topcard__title"}).text.strip()
    except AttributeError:
        pass

    try:
        company_tag = job_soup.find("a", {"class": "topcard__org-name-link"})
        job_post["Empresa"] = company_tag.text.strip()
        job_post["Link Empresa"] = company_tag.get("href")
    except AttributeError:
        pass

    try:
        criterios = job_soup.find_all("span", {"class": "description__job-criteria-text"})
        if len(criterios) > 0:
            job_post["Rubro"] = criterios[3].text.strip()
    except AttributeError:
        pass

    try:
        recruiter_tag = job_soup.find("a", {"class": "base-card__full-link"})
        job_post["Nombre del Reclutador"] = recruiter_tag.text.strip()
        job_post["Link Reclutador"] = recruiter_tag.get("href")
    except AttributeError:
        pass

    # Agregar el trabajo a la lista
    job_list.append(job_post)

# Ordenar trabajos por fecha de publicación y limitar a 30
job_list = sorted(job_list, key=lambda x: x["Fecha de Postulación"], reverse=True)[:70]

# Guardar los resultados en un archivo Excel
jobs_df = pd.DataFrame(job_list)
jobs_df.to_excel("LinkedIn_Recent_Jobs_Organized.xlsx", index=False, engine="openpyxl")

print("Se han guardado los resultados en LinkedIn_automatizacion_henry.xlsx")


Se encontraron 80 IDs únicos.
Se han guardado los resultados en LinkedIn_Recent_Jobs_Organized.xlsx


In [5]:
# Create a pandas DataFrame using the list of job dictionaries 'job_list'
jobs_df = pd.DataFrame(job_list)
jobs_df

,Estado,Posición,Fecha de Postulación,Plataforma,Empresa,Rubro,Nombre del Reclutador,Link Posición,Link Empresa,Link Reclutador
0,Postulado,"Data Analyst, Customer Operations",2025-01-15,LinkedIn,Squarespace,"Software Development and Technology, Informati...",None,https://www.linkedin.com/jobs/view/4097008893,https://www.linkedin.com/company/squarespace?t...,None
1,Postulado,Data Analyst,2025-01-15,LinkedIn,ANSER,Defense and Space Manufacturing,None,https://www.linkedin.com/jobs/view/4106857310,https://www.linkedin.com/company/analytic-serv...,None
2,Postulado,"Operations, Data Analyst",2025-01-15,LinkedIn,Oscar Health,Insurance,None,https://www.linkedin.com/jobs/view/4119999379,https://www.linkedin.com/company/oscar-health?...,None
3,Postulado,Business Data Analyst,2025-01-15,LinkedIn,Incedo Inc.,"Banking, Investment Banking, and Financial Ser...",None,https://www.linkedin.com/jobs/view/4126109722,https://www.linkedin.com/company/incedo-inc?tr...,None
4,Postulado,Program Data Analyst,2025-01-15,LinkedIn,Equiliem,Staffing and Recruiting,None,https://www.linkedin.com/jobs/view/4112146052,https://www.linkedin.com/company/equiliem?trk=...,None
5,Postulado,Senior Data Analyst,2025-01-15,LinkedIn,DHL Supply Chain,"Technology, Information and Media, Transportat...",None,https://www.linkedin.com/jobs/view/4108480450,https://de.linkedin.com/company/dhl-supply-cha...,None
6,Postulado,Data Analyst,2025-01-15,LinkedIn,Anthos|Home,Non-profit Organizations,None,https://www.linkedin.com/jobs/view/4125461097,https://www.linkedin.com/company/anthos-home?t...,None
7,Postulado,Data Analyst,2025-01-15,LinkedIn,ZBD,"Technology, Information and Internet",None,https://www.linkedin.com/jobs/view/4121192850,https://www.linkedin.com/company/zebedeeio?trk...,None
8,Postulado,Data Analyst,2025-01-15,LinkedIn,CloudIQ Technologies,IT Services and IT Consulting,Vinanthi Venkatesh,https://www.linkedin.com/jobs/view/4122077622,https://www.linkedin.com/company/cloud-iq-tech...,https://www.linkedin.com/in/vinanthi-venkatesh...
9,Postulado,Data Analyst,2025-01-15,LinkedIn,AdvantEdge,IT Services and IT Consulting and Internet Mar...,Venkat J Swaroop,https://www.linkedin.com/jobs/view/4123390968,https://www.linkedin.com/company/advantedgets?...,https://www.linkedin.com/in/venkatjyothiswaroop
